In [1]:
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
img_list = []
mask_list = []

for i in os.listdir('/kaggle/input/chasedb1'):
    if '.jpg' in i:
        img_list.append('/kaggle/input/chasedb1/'+i)
    else:
        if '1stHO' in i:
            mask_list.append('/kaggle/input/chasedb1/'+i)
            
img_list.sort()
mask_list.sort()
print("Done")

In [5]:
class RetinalDataset(Dataset):
    def __init__(self, images_list, masks_list, transform=None):
        self.transform = transform
        self.images_list = images_list
        self.masks_list = masks_list

    def __len__(self):
        return len(self.images_list)

    def __getitem__(self, idx):
        image = np.array(Image.open(self.images_list[idx]).convert('RGB')).astype(np.float32)
        mask = np.array(Image.open(self.masks_list[idx])).astype(np.uint8)

        # Center Crop
        height, width = image.shape[:2]
        crop_size = 960
        top = (height - crop_size) // 2
        left = (width - crop_size) // 2
        image = image[top:top+crop_size, left:left+crop_size]
        mask = mask[top:top+crop_size, left:left+crop_size]
        mask = mask.reshape((960, 960, 1))

        # Apply transformations
        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image, mask = augmented['image'], augmented['mask']

        image = image.astype(np.float32) 
        mask = mask.astype(np.float32) 
        mask = torch.tensor(mask).permute(2, 0, 1) 
        image = torch.tensor(image).permute(2, 0, 1)  
        return image, mask

transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.RandomBrightnessContrast(p=0.2, brightness_limit=0.2, contrast_limit=0.2),
    A.Normalize(mean=(0.5,), std=(0.5,)),
])

dataset = RetinalDataset(img_list, mask_list, transform=transform)
train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=123)
train_loader = DataLoader(train_data, batch_size=2, shuffle=True)
val_loader = DataLoader(val_data, batch_size=2, shuffle=False)
print('Done')

In [7]:
import torch
import torch.nn as nn

class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super(UNet, self).__init__()

        def conv_block(in_c, out_c):
            return nn.Sequential(
                nn.Conv2d(in_c, out_c, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_c, out_c, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
            )

        self.encoder1 = conv_block(in_channels, 64)
        self.encoder2 = conv_block(64, 128)
        self.encoder3 = conv_block(128, 256)
        self.encoder4 = conv_block(256, 512)
        self.bottleneck = conv_block(512, 1024)

        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.decoder4 = conv_block(1024, 512)
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.decoder3 = conv_block(512, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.decoder2 = conv_block(256, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.decoder1 = conv_block(128, 64)

        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        # Encoder
        e1 = self.encoder1(x)
        e2 = self.encoder2(nn.MaxPool2d(2)(e1))
        e3 = self.encoder3(nn.MaxPool2d(2)(e2))
        e4 = self.encoder4(nn.MaxPool2d(2)(e3))
        b = self.bottleneck(nn.MaxPool2d(2)(e4))

        # Decoder
        d4 = self.upconv4(b)
        d4 = self.decoder4(torch.cat([d4, e4], dim=1))
        d3 = self.upconv3(d4)
        d3 = self.decoder3(torch.cat([d3, e3], dim=1))
        d2 = self.upconv2(d3)
        d2 = self.decoder2(torch.cat([d2, e2], dim=1))
        d1 = self.upconv1(d2)
        d1 = self.decoder1(torch.cat([d1, e1], dim=1))

        return torch.sigmoid(self.final_conv(d1))

print(True)

True


In [9]:
def dice_loss(pred, target):
    smooth = 1.0
    pred = pred.contiguous()
    target = target.contiguous()
    intersection = (pred * target).sum(dim=2).sum(dim=2)
    loss = 1 - ((2. * intersection + smooth) / (pred.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + smooth))
    return loss.mean()

print(True)

True


In [10]:
model = UNet().cuda()
criterion_bce = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

best_train_loss = float('inf')
best_model_path = "best_model.pth"  
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    
    for images, masks in tqdm(train_loader):
        
        images = images.cuda()
        masks = masks.cuda()
        optimizer.zero_grad()
        outputs = model(images)
        total_loss = criterion_bce(outputs, masks) + dice_loss(outputs, masks)
        total_loss.backward()
        optimizer.step()
        train_loss += total_loss.item()
        
    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_train_loss:.4f}")
    
    if avg_train_loss < best_train_loss:
        best_train_loss = avg_train_loss
        torch.save(model.state_dict(), best_model_path)
        print(f"Model saved with train loss: {best_train_loss:.4f}")


100%|██████████| 11/11 [00:16<00:00,  1.48s/it]


Epoch 1/50, Loss: 1.5314
Model saved with train loss: 1.5314


100%|██████████| 11/11 [00:15<00:00,  1.42s/it]


Epoch 2/50, Loss: 1.3866
Model saved with train loss: 1.3866


100%|██████████| 11/11 [00:16<00:00,  1.46s/it]


...

Epoch 48/50, Loss: 0.5128


100%|██████████| 11/11 [00:18<00:00,  1.68s/it]


Epoch 49/50, Loss: 0.4989
Model saved with train loss: 0.4989


100%|██████████| 11/11 [00:18<00:00,  1.68s/it]

Epoch 50/50, Loss: 0.5017


In [13]:
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    
    for images, masks in tqdm(train_loader):
        
        images = images.cuda()
        masks = masks.cuda()
        optimizer.zero_grad()
        outputs = model(images)
        total_loss = criterion_bce(outputs, masks) + dice_loss(outputs, masks)
        total_loss.backward()
        optimizer.step()
        train_loss += total_loss.item()
        
    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch+1+50}/{num_epochs+50}, Loss: {avg_train_loss:.4f}")
    
    if avg_train_loss < best_train_loss:
        best_train_loss = avg_train_loss
        torch.save(model.state_dict(), best_model_path)
        print(f"Model saved with train loss: {best_train_loss:.4f}")

100%|██████████| 11/11 [00:16<00:00,  1.46s/it]


Epoch 51/100, Loss: 0.5222


100%|██████████| 11/11 [00:16<00:00,  1.48s/it]


Epoch 52/100, Loss: 0.5555


100%|██████████| 11/11 [00:16<00:00,  1.51s/it]


...

Epoch 98/100, Loss: 0.4620
Model saved with train loss: 0.4620


100%|██████████| 11/11 [00:18<00:00,  1.68s/it]


Epoch 99/100, Loss: 0.4466
Model saved with train loss: 0.4466


100%|██████████| 11/11 [00:18<00:00,  1.69s/it]

Epoch 100/100, Loss: 0.4551


In [19]:
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    
    for images, masks in tqdm(train_loader):
        
        images = images.cuda()
        masks = masks.cuda()
        optimizer.zero_grad()
        outputs = model(images)
        total_loss = criterion_bce(outputs, masks) + dice_loss(outputs, masks)
        total_loss.backward()
        optimizer.step()
        train_loss += total_loss.item()
        
    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch+1+100}/{num_epochs+100}, Loss: {avg_train_loss:.4f}")
    
    if avg_train_loss < best_train_loss:
        best_train_loss = avg_train_loss
        torch.save(model.state_dict(), best_model_path)
        print(f"Model saved with train loss: {best_train_loss:.4f}")

100%|██████████| 11/11 [00:16<00:00,  1.47s/it]


Epoch 101/150, Loss: 0.4714


100%|██████████| 11/11 [00:16<00:00,  1.49s/it]


Epoch 102/150, Loss: 0.4820


100%|██████████| 11/11 [00:16<00:00,  1.51s/it]


...

Epoch 148/150, Loss: 0.3727


100%|██████████| 11/11 [00:18<00:00,  1.69s/it]


Epoch 149/150, Loss: 0.3701
Model saved with train loss: 0.3701


100%|██████████| 11/11 [00:18<00:00,  1.69s/it]


Epoch 150/150, Loss: 0.3694
Model saved with train loss: 0.3694


In [20]:
from IPython.display import FileLink
FileLink(r'best_model.pth')

/kaggle/working/best_model.pth

In [21]:
def evaluate_model(model, val_loader):
    model.eval()
    accuracies = []
    f1_scores = []
    with torch.no_grad():
        for images, masks in val_loader:
            images = images.cuda()
            masks = masks.cuda()
            outputs = model(images)
            preds = (outputs > 0.5).float()

            # Flatten predictions and masks
            preds_flat = preds.view(-1)
            masks_flat = masks.view(-1)

            # True Positives, True Negatives, False Positives, False Negatives
            TP = (preds_flat * masks_flat).sum().item()
            TN = ((1 - preds_flat) * (1 - masks_flat)).sum().item()
            FP = (preds_flat * (1 - masks_flat)).sum().item()
            FN = ((1 - preds_flat) * masks_flat).sum().item()

            # Accuracy
            accuracy = (TP + TN) / (TP + TN + FP + FN)
            accuracies.append(accuracy)

            # Precision and Recall for F1 Score
            precision = TP / (TP + FP + 1e-7)  # Avoid division by zero
            recall = TP / (TP + FN + 1e-7)    # Avoid division by zero
            f1_score = 2 * (precision * recall) / (precision + recall + 1e-7)  # F1 score
            f1_scores.append(f1_score)

    # Print mean metrics
    print(f"Mean Accuracy: {np.mean(accuracies):.4f}")
    print(f"Mean F1 Score: {np.mean(f1_scores):.4f}")

evaluate_model(model, val_loader)

Mean Accuracy: 0.9634
Mean F1 Score: 0.7179
